<a href="https://colab.research.google.com/github/tomonari-masada/course2021-nlp/blob/main/Assignment_20211127.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(123)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
PATH = '/content/drive/MyDrive/2021Courses/NLP/'

texts = dict()
labels = dict()
for tag in ['train', 'test']:
  with open(f'{PATH}{tag}.npy', 'rb') as f:
    texts[tag] = np.load(f)
  with open(f'{PATH}{tag}_labels.npy', 'rb') as f:
    labels[tag] = np.load(f)

In [3]:
for tag in ['train', 'test']:
  texts[tag], labels[tag] = torch.tensor(texts[tag]), torch.tensor(labels[tag])

In [4]:
from torch.utils.data import Dataset, random_split

class MyDataset(Dataset):
  def __init__(self, X, y):
    self.X = X
    self.y = y

  def __len__(self):
    return self.X.shape[0]

  def __getitem__(self, index):
    return self.X[index], self.y[index]

train_valid = MyDataset(texts['train'], labels['train'])
test = MyDataset(texts['test'], labels['test'])

valid_size = len(train_valid) // 5
train_size = len(train_valid) - valid_size
train, valid = random_split(train_valid,
                            [train_size, valid_size],
                            generator=torch.Generator().manual_seed(42)
                            )

In [5]:
from torch.utils.data import DataLoader

BATCH_SIZE = 100

train_loader = DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid, batch_size=BATCH_SIZE)
test_loader = DataLoader(test, batch_size=BATCH_SIZE)

In [6]:
EMBED_DIM = texts['train'].size(1)
NUM_CLASS = len(np.unique(labels['train']))


In [7]:
def eval(model, criterion, loader):
  model.eval()
  
  total_loss = 0.0
  total_acc = 0.0
  total_size = 0
  for input, target in loader:
    with torch.no_grad():
      input, target = input.to(device), target.to(device)
      output = model(input)
      loss = criterion(output, target)
      total_loss += loss.item() * len(target)
      total_acc += (output.argmax(1) == target).float().sum().item()
      total_size += len(target)

  return total_loss / total_size, total_acc / total_size

In [8]:
def train(model, criterion, optimizer, train_loader, valid_loader, n_epochs=100, scheduler=None):
  model.train()

  # training loop
  for epoch in range(n_epochs):

    train_loss = 0.0
    for input, target in train_loader:
      output = model(input.to(device))
      loss = criterion(output, target.to(device))
      train_loss += loss.item() * len(target) # 表示用の集計

      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

    if scheduler:
      scheduler.step()

    valid_loss, valid_acc = eval(model, criterion, valid_loader)

    # logging
    print(f'epoch {epoch + 1:6d} |',
          f'train loss {train_loss / train_size:8.4f} |',
          f'valid loss {valid_loss:8.4f} | valid acc {valid_acc:8.3f}')

In [9]:
class TextSentiment(nn.Module):
  def __init__(self, embed_dim, num_class, hidden_dim=500):
    super(TextSentiment, self).__init__()
    self.fc1 = nn.Linear(embed_dim, hidden_dim)
    self.fc2 = nn.Linear(hidden_dim, hidden_dim)
    self.fc3 = nn.Linear(hidden_dim, hidden_dim)
    self.out = nn.Linear(hidden_dim, num_class)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = self.out(x)
    return x

In [10]:
model = TextSentiment(EMBED_DIM, NUM_CLASS, 500).to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50], gamma=0.1)

In [11]:
train(model, criterion, optimizer, train_loader, valid_loader, 100, scheduler=scheduler)

epoch      1 | train loss   0.6494 | valid loss   0.5119 | valid acc    0.763
epoch      2 | train loss   0.4262 | valid loss   0.4141 | valid acc    0.815
epoch      3 | train loss   0.3811 | valid loss   0.3865 | valid acc    0.834
epoch      4 | train loss   0.3644 | valid loss   0.3711 | valid acc    0.842
epoch      5 | train loss   0.3560 | valid loss   0.3703 | valid acc    0.843
epoch      6 | train loss   0.3523 | valid loss   0.3616 | valid acc    0.847
epoch      7 | train loss   0.3457 | valid loss   0.3627 | valid acc    0.848
epoch      8 | train loss   0.3421 | valid loss   0.3575 | valid acc    0.850
epoch      9 | train loss   0.3408 | valid loss   0.3643 | valid acc    0.843
epoch     10 | train loss   0.3396 | valid loss   0.3606 | valid acc    0.843
epoch     11 | train loss   0.3362 | valid loss   0.3536 | valid acc    0.851
epoch     12 | train loss   0.3342 | valid loss   0.3679 | valid acc    0.837
epoch     13 | train loss   0.3332 | valid loss   0.3512 | valid

In [12]:
loss, acc = eval(model, criterion, test_loader)
print(f'test loss {loss:8.4f} | test acc {acc:8.3f}')

test loss   0.3410 | test acc    0.856
